In [2]:
import os
import sys
PROJECT_ROOT = os.path.abspath(os.curdir) ## Please make sure it outputs the correct root, No universal solution for Colab - Jupyter - Local environment

print('The Project is running from : ', PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)
 

The Project is running from :  /content/drive/MyDrive/MyFinal-ML-Folder0


In [3]:
##Imports 
import Loading_helpers
import Model
import Predictions_Writer
import Tuning

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models,transforms
import cv2
from PIL import Image

import unet.unet_depth_3
import unet.unet_depth_4
import unet.unet_depth_5


In [4]:
# Training Constants 
BATCH_SIZE = 6
NB_EPOCHS = 75
LEARNING_RATE = 0.0002
SIGMOID_THRESHOLD_TRAINING = 0.50

## MODELS_DICT takes the format {name of the model : [Constructor,Where the trained model can be loaded from]}
MODELS_DICTIONARY = {"UNET DEPTH 3" :[unet.unet_depth_3.UNet(3,1),PROJECT_ROOT+'/Unet_depth_3.pth'],
                      "UNET DEPTH 4" :[unet.unet_depth_4.UNet(3,1),PROJECT_ROOT+'/Unet_depth_4.pth'],
                      "UNET DEPTH 5" :[unet.unet_depth_5.UNet(3,1),PROJECT_ROOT+'/Unet_depth_5.pth'],
                      "DEEPLABV3-RESNET101" : [models.segmentation.deeplabv3_resnet101(pretrained=False,num_classes=1),PROJECT_ROOT+'/deepLabV3_resnet_101.pth'],
                      "DEEPLABV3-MOBILENETV3-LARGE" :[models.segmentation.deeplabv3_mobilenet_v3_large(pretrained=False,num_classes=1),PROJECT_ROOT+'/deepLabV3_MobileNetV3_large.pth'],
                      "LRASPP-MOBILENETV3-LARGE": [models.segmentation.lraspp_mobilenet_v3_large(pretrained=False,num_classes=1),PROJECT_ROOT+'/lraspp_MobileNetV3_large.pth']}

## Set TO TRUE if you want the model to be loaded
MODEL_ALREADY_TRAINED = True
## Set to True if you want to use the fulldataset for training, no validation set.
FULL_DATASET = True
## Select the model you want to try
MODEL_TO_TRY = "DEEPLABV3-RESNET101"
MODEL_DETAILS = MODELS_DICTIONARY[MODEL_TO_TRY]
## Set to True if you want to save the model recently trained. 
SAVE_MODEL = False 


# Hyperparameters Tuning Constants
HYPERPARAMETERS_TUNING = False

# Prediction Constants
THRESHOLD_ROAD_LABEL_PREDICTION = 0.4

In [5]:
def main():

    ## Loading Phase
    train_data, validation_data, train_label, validation_label = Loading_helpers.load(split= not FULL_DATASET)

    train_dataset = Loading_helpers.Image_Dataset(train_data,train_label,transformation=True,normalize = transforms.Normalize(mean=[0, 0,0],std=[1,1,1]))
    train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=2)
    
    if(FULL_DATASET):
      validation_dataset = None
      validation_loader = None
    else :
      validation_dataset = Loading_helpers.Image_Dataset(validation_data,validation_label,normalize = transforms.Normalize(mean=[0,0,0],std=[1,1,1]))
      validation_loader = DataLoader(dataset=validation_dataset,
                         batch_size=BATCH_SIZE,
                          shuffle=True,
                         num_workers=2)
    
    ## Training Phase 

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Running on ', device)
    model = MODEL_DETAILS[0]
    
    if(MODEL_ALREADY_TRAINED):
      model.load_state_dict(torch.load(MODEL_DETAILS[1]))
      model=model.to(device)
    else :
      model=model.to(device)
      model,_ = Model.train_model(model,NB_EPOCHS,train_loader,validation_loader,device,LEARNING_RATE,
                                SIGMOID_THRESHOLD_TRAINING,model_name = MODEL_TO_TRY,saving_path = MODEL_DETAILS[1],
                                optimized_for_AICrowd = FULL_DATASET, save_model = SAVE_MODEL)
      
    ## Hyperparameters Tuning Phase 
    if(HYPERPARAMETERS_TUNING):
      analysis, best_hyp= Tuning.find_best_params(model,train_loader,validation_loader,device,'Grid Search')

    ## Prediction Phase 
    Predictions_Writer.generate_predictions(model,THRESHOLD_ROAD_LABEL_PREDICTION,device,model_name=MODEL_TO_TRY)
    
    
    print('End of the program')
    
    



In [6]:
main()

Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_001.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_002.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_003.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_004.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_005.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_006.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_007.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_008.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_009.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_010.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_011.png
Loading /content/drive/MyDrive/MyFinal-ML-Folder0/training/images/satImage_012.png
Load